# Projekt 1.

## Przygotowanie danych.

In [30]:
import pandas as pd
import numpy as np

dane = pd.read_excel("loty_clean.xlsx")

dane.head()

,Extraction_Time,Extraction_Weekday,Flight_weekday,Flight_date,Departure_time,Arrival_time,Flight_time,Departure_airport_code,arr_city,dep_city,...,Cabin_bag,Checked_bag,Days_to_departure,layover_airport,layover_duration,Airline1,low_cost1,Airline2,low_cost2,ujemne
0,02/11/2024 19:16,Saturday,Monday,2024-11-04,16.58,17.92,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
1,02/11/2024 19:16,Saturday,Monday,2024-11-04,18.33,22.67,4.33,WAW,Berlin,Warszawa,...,0,0,2,CPH,1.583333,Wizz Air,low-cost,easyJet,low-cost,-2
2,02/11/2024 19:16,Saturday,Monday,2024-11-04,19.67,21.00,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
3,02/11/2024 19:16,Saturday,Monday,2024-11-04,6.08,11.92,5.83,WAW,Berlin,Warszawa,...,0,0,2,FCO,1.083333,Wizz Air,low-cost,easyJet,low-cost,-2
4,02/11/2024 19:16,Saturday,Monday,2024-11-04,17.00,22.67,5.67,WAW,Berlin,Warszawa,...,1,0,2,MUC,2.833333,Lufthansa,legacy,Lufthansa,legacy,-2


Zamiana nazw kolumn.

In [31]:
dane = dane.rename(columns={"#Layovers": "Num_Layovers", "Price [PLN]": "Price"})
dane.columns

Index(['Extraction_Time', 'Extraction_Weekday', 'Flight_weekday',
       'Flight_date', 'Departure_time', 'Arrival_time', 'Flight_time',
       'Departure_airport_code', 'arr_city', 'dep_city',
       'Departure_airport_name', 'Destination_airport_code',
       'Destination_airport_name', 'Price', 'Num_Layovers', 'Ticket_class',
       'Cabin_bag', 'Checked_bag', 'Days_to_departure', 'layover_airport',
       'layover_duration', 'Airline1', 'low_cost1', 'Airline2', 'low_cost2',
       'ujemne'],
      dtype='object')

In [32]:
# przekształcenie dat
dane["Flight_date"] = pd.to_datetime(dane["Flight_date"])
dane["Extraction_Time"] = pd.to_datetime(dane["Extraction_Time"].str.split(" ").apply(lambda x: x[0]), dayfirst=True)


dane.head()

,Extraction_Time,Extraction_Weekday,Flight_weekday,Flight_date,Departure_time,Arrival_time,Flight_time,Departure_airport_code,arr_city,dep_city,...,Cabin_bag,Checked_bag,Days_to_departure,layover_airport,layover_duration,Airline1,low_cost1,Airline2,low_cost2,ujemne
0,2024-11-02,Saturday,Monday,2024-11-04,16.58,17.92,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
1,2024-11-02,Saturday,Monday,2024-11-04,18.33,22.67,4.33,WAW,Berlin,Warszawa,...,0,0,2,CPH,1.583333,Wizz Air,low-cost,easyJet,low-cost,-2
2,2024-11-02,Saturday,Monday,2024-11-04,19.67,21.00,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
3,2024-11-02,Saturday,Monday,2024-11-04,6.08,11.92,5.83,WAW,Berlin,Warszawa,...,0,0,2,FCO,1.083333,Wizz Air,low-cost,easyJet,low-cost,-2
4,2024-11-02,Saturday,Monday,2024-11-04,17.00,22.67,5.67,WAW,Berlin,Warszawa,...,1,0,2,MUC,2.833333,Lufthansa,legacy,Lufthansa,legacy,-2


In [33]:
# usunięcie zbędnych (czy na pewno?) kolumn
# "arr_city", "dep_city",  - co zrobić?
dane = dane.loc[dane["low_cost1"] == dane["low_cost2"], ]
dane.drop(columns = ["Extraction_Time", "Flight_date", "Departure_airport_code", "Destination_airport_code",
                     "Departure_airport_name", "Destination_airport_name", 
                     "layover_airport", "ujemne", "low_cost2", "Airline2", "Airline1", "Ticket_class"], inplace=True)

In [34]:
onehot_departure = pd.get_dummies(dane["dep_city"], prefix= "Is_Departure", drop_first=True)
onehot_arrival = pd.get_dummies(dane, columns=["arr_city"], prefix= "Is_Arrival", drop_first=True)

onehot_flight_weekday = pd.get_dummies(dane["Flight_weekday"], prefix = "Is_Flight", drop_first=True)
onehot_extraction_weekday = pd.get_dummies(dane["Extraction_Weekday"], prefix = "Is_Extraction", drop_first=True)

onehot_lowcost = pd.get_dummies(dane["low_cost1"], prefix = "Is", drop_first=True)

# onehot_airline2 = pd.get_dummies(dane["Airline2"], prefix="Is", drop_first=False)
# onehot_airline2.columns = [x + "2" for x in onehot_airline2.columns]


dane = pd.concat([dane, onehot_departure, onehot_arrival, onehot_flight_weekday, onehot_extraction_weekday, onehot_lowcost], axis=1)

dane.columns

Index(['Extraction_Weekday', 'Flight_weekday', 'Departure_time',
       'Arrival_time', 'Flight_time', 'arr_city', 'dep_city', 'Price',
       'Num_Layovers', 'Cabin_bag', 'Checked_bag', 'Days_to_departure',
       'layover_duration', 'low_cost1', 'Is_Departure_Warszawa',
       'Extraction_Weekday', 'Flight_weekday', 'Departure_time',
       'Arrival_time', 'Flight_time', 'dep_city', 'Price', 'Num_Layovers',
       'Cabin_bag', 'Checked_bag', 'Days_to_departure', 'layover_duration',
       'low_cost1', 'Is_Arrival_Londyn', 'Is_Arrival_Paryż', 'Is_Arrival_Rzym',
       'Is_Flight_Monday', 'Is_Flight_Saturday', 'Is_Flight_Sunday',
       'Is_Flight_Thursday', 'Is_Flight_Tuesday', 'Is_Flight_Wednesday',
       'Is_Extraction_Monday', 'Is_Extraction_Saturday',
       'Is_Extraction_Sunday', 'Is_Extraction_Thursday',
       'Is_Extraction_Tuesday', 'Is_Extraction_Wednesday', 'Is_low-cost'],
      dtype='object')

Usuwamy kolumny zbędne po onehotencodingu.

In [35]:
dane.shape

(105491, 44)

In [36]:
dane.drop(columns = ["low_cost1", "dep_city", "arr_city", "Flight_weekday", 
                      "Extraction_Weekday"], inplace=True)



In [37]:
dane.columns

Index(['Departure_time', 'Arrival_time', 'Flight_time', 'Price',
       'Num_Layovers', 'Cabin_bag', 'Checked_bag', 'Days_to_departure',
       'layover_duration', 'Is_Departure_Warszawa', 'Departure_time',
       'Arrival_time', 'Flight_time', 'Price', 'Num_Layovers', 'Cabin_bag',
       'Checked_bag', 'Days_to_departure', 'layover_duration',
       'Is_Arrival_Londyn', 'Is_Arrival_Paryż', 'Is_Arrival_Rzym',
       'Is_Flight_Monday', 'Is_Flight_Saturday', 'Is_Flight_Sunday',
       'Is_Flight_Thursday', 'Is_Flight_Tuesday', 'Is_Flight_Wednesday',
       'Is_Extraction_Monday', 'Is_Extraction_Saturday',
       'Is_Extraction_Sunday', 'Is_Extraction_Thursday',
       'Is_Extraction_Tuesday', 'Is_Extraction_Wednesday', 'Is_low-cost'],
      dtype='object')

Konwersja całego dataframe na typ numeric i podział na zbiory treningowe i testowe oraz wektor celu i macierz cech.

In [38]:
from sklearn.model_selection import train_test_split
dane = dane.apply(pd.to_numeric)
dane.to_pickle("dane_klasyfikacja_onehot2.pkl")